# Distribution Monad - Forkmann

http://www.navision-blog.de/blog/2011/08/15/some-special-monads-in-f-part-3-of-n-distributionmonad/

Martin Erwig and Steve Kollmansberger "Functional Pearls: Probabilistic functional programming in Haskell". http://web.engr.oregonstate.edu/~erwig/papers/PFP_JFP06.pdf

#### `Outcome`

In [2]:
type  Outcome<'a> = {
    Value: 'a
    Probability : double  }

#### `Distribution`

In [3]:
type Distribution<'a> = Outcome<'a> seq

#### `bindD`

 (aka `flatMap`)

In [4]:
// P(A AND B) = P(A | B) * P(B)
let bindD (dist: Distribution<'b>) (f: 'b -> Distribution<'a >) =
    dist 
        |> Seq.map (fun p1 -> 
                f p1.Value
                |> Seq.map (fun p2 -> 
                        { Value = p2.Value; 
                            Probability = 
                              p1.Probability * p2.Probability}))
        |> Seq.concat : Distribution<'a >

#### `returnD`

In [6]:
let returnD (value:'a) : Distribution<'a > =   
    Seq.singleton { Value = value ; Probability = 1.0 }

#### `DistributionMonadBuilder`

Define a computation expression builder:

In [7]:
type DistributionMonadBuilder() =
    member this.Bind (r, f) = bindD r f
    member this.Return x = returnD x
    member this.ReturnFrom m = m

#### `distribution`

In [9]:
let distribution = DistributionMonadBuilder()

#### Test

In [ ]:
let fairCoinAndDice = distribution {
    let! d = fairDice 6
    let! c = fairCoin
    return d,c }

In [ ]:
fairCoinAndDice
  |> filter (fun (d,c) -> c = Heads && d > 3)
  |> probability
  |> printfn "P(Heads and dice > 3) = %A" // "1/4N"